In [34]:
import argparse
import time
import cv2
import math
import numpy as np
from pyzbar import pyzbar
import imutils

In [35]:
def distance(point_x, point_y):
    """ Calculate distance between two points. """
    return math.sqrt(math.pow(math.fabs(point_x[0]-point_y[0]), 2) + math.pow(math.fabs(point_x[1]-point_y[1]), 2))

In [36]:
import sys
def line_equation(point_l, point_m, point_j):
    """ Calculate distance between line (passing thorugh point l and point m) and point j """
    
    # We calculate coefficients 'a' for line passing through points l and m thus we calculate 'c' by
    # using formula: y = ax + c =  > c = ax - y """
    dif = (point_m[0] - point_l[0])
    if dif == 0: dif = sys.maxsize
        
    coef_a = -((point_m[1] - point_l[1])/dif)
    coef_b = 1.0
    # We use ax + by + c = 0 reprezentation, beacuse it's then we check if a point is on line,
    # or if it is above or below line.
    coef_c = (((point_m[1] - point_l[1])/dif)
              * point_l[0]) - point_l[1]
    try:
        # calculate ax + by + c, the value is one cathetus ofrectangular triangle,
        # the second is perpendicular to line, and therefore it's the distance form the
        # point to line, we calculate it by divide hypotenuse by cathetus
        pdist = (coef_a*point_j[0]+(coef_b*point_j[1]) +
                 coef_c)/math.sqrt((coef_a*coef_a)+(coef_b*coef_b))
    except():
        return 0
    else:
        return pdist



In [37]:
def line_slope(l, m):
    ''' Wspólczynnik kierunkowy / pochodna funkcji. Slope/ Implementation of function derivative dy/dx'''
    dx = m[0] - l[0]
    dy = m[1] - l[1]
    if dy != 0:
        align = 1 
        dxy = dy/dx
        return dxy, align
    else:
        align = 0
        dxy = 0.0
        return dxy, align

In [38]:
def get_squares(contours, cid):
    x, y, w, h = cv2.boundingRect(contours[cid])
    return x, y, w, h

In [39]:
def update_corner(p, ref, baseline, corner):
    '''Update given baseline and corner if distance from p to red is greater'''
    temp_dist = distance(p, ref)
    if temp_dist > baseline:
        baseline = temp_dist
        corner = p
    return baseline, corner

In [40]:
def get_vertices(contours, cid, slope, quad):
    ''' Foreach contours return verticies of each rect closest to contour blablah..'''
    M0 = (0.0, 0.0)
    M1 = (0.0, 0.0)
    M2 = (0.0, 0.0)
    M3 = (0.0, 0.0)
    x, y, w, h = cv2.boundingRect(contours[cid])
    A = (x, y) # Bottom left
    B = (x+w, y) #  Bottom righnt
    C = (x+w, h+y) # top right
    D = (x, y+h) # Top left
    W = ((A[0]+B[0])/2, A[1])
    X = (B[0], (B[1]+C[1])/2)
    Y = ((C[0]+D[0])/2, C[1])
    Z = (D[0], (D[1]+A[1])/2)
    dmax = []
    for i in range(4):
        dmax.append(0.0)
    pd1 = 0.0
    pd2 = 0.0
    if(slope > 5 or slope < -5): # atan(1)->45deg, atan(5) -> 78deg
        for i in range(len(contours[cid])):
            pd1 = line_equation(C, A, contours[cid][i][0])
            pd2 = line_equation(B, D, contours[cid][i][0])
            if(pd1 >= 0.0 and pd2 > 0.0): # Dolny Prawy
                dmax[1], M1 = update_corner(contours[cid][i][0], W, dmax[1], M1)
            elif(pd1 > 0.0 and pd2 <= 0): # Dolny Lewy
                dmax[2], M2 = update_corner(contours[cid][i][0], X, dmax[2], M2)
            elif(pd1 <= 0.0 and pd2 < 0.0): # Górny Lewy
                dmax[3], M3 = update_corner(contours[cid][i][0], Y, dmax[3], M3)
            elif(pd1 < 0 and pd2 >= 0.0): #Górny Prawy
                dmax[0], M0 = update_corner(contours[cid][i][0], Z, dmax[0], M0)
            else:
                continue
    else:
        halfx = (A[0]+B[0])/2
        halfy = (A[1]+D[1])/2
        for i in range(len(contours[cid])):
            if(contours[cid][i][0][0] < halfx and contours[cid][i][0][1] <= halfy):
                dmax[2], M0 = update_corner(
                    contours[cid][i][0], C, dmax[2], M0)
            elif(contours[cid][i][0][0] >= halfx and contours[cid][i][0][1] < halfy):
                dmax[3], M1 = update_corner(
                    contours[cid][i][0], D, dmax[3], M1)
            elif(contours[cid][i][0][0] > halfx and contours[cid][i][0][1] >= halfy):
                dmax[0], M2 = update_corner(
                    contours[cid][i][0], A, dmax[0], M2)
            elif(contours[cid][i][0][0] <= halfx and contours[cid][i][0][1] > halfy):
                dmax[1], M3 = update_corner(
                    contours[cid][i][0], B, dmax[1], M3)
    quad.append(M0)
    quad.append(M1)
    quad.append(M2)
    quad.append(M3)
    return quad

In [41]:
def update_corner_or(orientation, IN):
    if orientation == 0:
        M0 = IN[0]
        M1 = IN[1]
        M2 = IN[2]
        M3 = IN[3]
    elif orientation == 1:
        M0 = IN[1]
        M1 = IN[2]
        M2 = IN[3]
        M3 = IN[0]
    elif orientation == 2:
        M0 = IN[2]
        M1 = IN[3]
        M2 = IN[0]
        M3 = IN[1]
    elif orientation == 3:
        M0 = IN[3]
        M1 = IN[0]
        M2 = IN[1]
        M3 = IN[2]
        
    OUT = []
    OUT.append(M0)
    OUT.append(M1)
    OUT.append(M2)
    OUT.append(M3)

    return OUT

In [42]:
def cross(v1, v2):
    cr = v1[0]*v2[1] - v1[1]*v2[0]
    return cr

In [43]:
def get_intersection(a1, a2, b1, b2, intersection):
    p = a1
    q = b1
    r = (a2[0]-a1[0], a2[1]-a1[1])
    s = (b2[0]-b1[0], b2[1]-b1[1])
    if cross(r, s) == 0:
        return False, intersection
    t = cross((q[0]-p[0], q[1]-p[1]), s)/float(cross(r, s))
    intersection = (int(p[0]+(t*r[0])), int(p[1]+(t*r[1])))
    return True, intersection

In [44]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

In [45]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    dst = np.array([
        [0, 0],
        [319, 0],
        [319, 319],
        [0, 319]], dtype="float32")
    
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (320, 320))  # maxWidth, maxHeight))

    # return the warped image
    return warped

In [46]:
cv2.namedWindow('rect')
cap = cv2.VideoCapture(1)

# Reduce the size of video to 320x240 so rpi can process faster
# cap.set(3,640)
# cap.set(4,480)

# camera = PiCamera()
# camera.resolution = (640,480)
# camera.framerate = 32
# rawCapture = PiRGBArray(camera,size=(640,480))
# time.sleep(0.1)
# for frame in camera.capture_continuous(rawCapture,format="bgr",use_video_port=True):
# image = frame.array
# img = image

# show the image
# wait until some key is pressed to procced
while True:
    _, image = cap.read()
    img = image
    edges = cv2.Canny(image, 100, 200)
    cv2.imshow("Canny", edges)
    # contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    im2, contours, hierarchy = cv2.findContours(
        edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    approxCnt = []
    for i, cnt in enumerate(contours):
        epsilon = 0.001*cv2.arcLength(cnt,True)
        approxCnt.append(cv2.approxPolyDP(cnt,epsilon,True))
        
    squares = [cnt for cnt in approxCnt if len(cnt) < 20]
    contours = squares
    
    colors = np.random.randint(0,255,(len(contours),3))
    color = [tuple([int(c) for c in color]) for color in colors]
    
    contImg = np.zeros((480,640,3), dtype="uint8")
    for i, cnt in enumerate(contours):
        cv2.drawContours(contImg, contours, i, color[i])
    cv2.imshow("contours", contImg)
    
    mu = []
    mc = []
    mark = 0
    for x in range(0, len(contours)):
        mu.append(cv2.moments(contours[x]))

    for m in mu:
        if m['m00'] != 0:
            mc.append((m['m10']/m['m00'], m['m01']/m['m00']))
        else:
            mc.append((0, 0))

    for x in range(0, len(contours)):
        k = x
        c = 0
        while(hierarchy[0][k][2] != -1):
            k = hierarchy[0][k][2]
            c = c + 1
        if hierarchy[0][k][2] != -1:
            c = c + 1

        if c >= 5:
            if mark == 0:
                A = x
            elif mark == 1:
                B = x
            elif mark == 2:
                C = x
            mark = mark+1

    if mark > 2:
        AB = distance(mc[A], mc[B])
        BC = distance(mc[B], mc[C])
        AC = distance(mc[A], mc[C])

        if(AB > BC and AB > AC):
            outlier = C
            median1 = A
            median2 = B
        elif(AC > AB and AC > BC):
            outlier = B
            median1 = A
            median2 = C
        elif(BC > AB and BC > AC):
            outlier = A
            median1 = B
            median2 = C

        top = outlier
        dist = line_equation(mc[median1], mc[median2], mc[outlier])
        slope, align = line_slope(mc[median1], mc[median2])

        if align == 0:
            bottom = median1
            right = median2
        elif(slope < 0 and dist < 0):
            bottom = median1
            right = median2
            orientation = 0
        elif(slope > 0 and dist < 0):
            right = median1
            bottom = median2
            orientation = 1
        elif(slope < 0 and dist > 0):
            right = median1
            bottom = median2
            orientation = 2
        elif(slope > 0 and dist > 0):
            bottom = median1
            right = median2
            orientation = 3

        areatop = 0.0
        arearight = 0.0
        areabottom = 0.0
        if(top < len(contours) and right < len(contours) and bottom < len(contours) 
           and cv2.contourArea(contours[top]) > 10 
           and cv2.contourArea(contours[right]) > 10 
           and cv2.contourArea(contours[bottom]) > 10):
            tempL = []
            tempM = []
            tempO = []
            src = []
            N = (0, 0)
            tempL = get_vertices(contours, top, slope, tempL)
            tempM = get_vertices(contours, right, slope, tempM)
            tempO = get_vertices(contours, bottom, slope, tempO)
            L = update_corner_or(orientation, tempL)
            M = update_corner_or(orientation, tempM)
            O = update_corner_or(orientation, tempO)

            iflag, N = get_intersection(M[1], M[2], O[3], O[2], N)
            src.append(L[0])
            src.append(M[1])
            src.append(N)
            src.append(O[3])
            src = np.asarray(src, np.float32)
            warped1 = four_point_transform(img, src)
            warped = warped1
            # sw added to rotate image to correct orientation for visual purposes only - not needed for zbar
            rowsrot, colsrot, dummy = warped1.shape
            # print rowsrot, colsrot, dummy
            if orientation > 0:
                Mrot = cv2.getRotationMatrix2D(
                    (colsrot/2, rowsrot/2), (90 * orientation), 1)
                warped = cv2.warpAffine(warped1, Mrot, (colsrot, rowsrot))
            cv2.imshow("warped", warped)
            cv2.circle(img, N, 1, (0, 0, 255), 2)
            cv2.drawContours(img, contours, top, (255, 0, 0), 2)
            cv2.drawContours(img, contours, right, (0, 255, 0), 2)
            cv2.drawContours(img, contours, bottom, (0, 0, 255), 2)
            warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            # scanner = zbar.ImageScanner()
            # scanner.parse_config('enable')
            # imagez = zbar.Image(
            #     warped.shape[0], warped.shape[1], 'Y800', warped.tostring())
            # scanner.scan(imagez)
            decoded = pyzbar.decode(warped)

            for symbol in decoded:
                x = symbol.data
                print(x)
                print(int(180.0 * slope / 3.1415926))

    cv2.imshow("rect", img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()

cv2.destroyAllWindows()


In [33]:
cv2.destroyAllWindows()
cap.release()

In [70]:
lenCnts = [len(c) for c in contours if True]

In [27]:
approxCnt = []
for i, cnt in enumerate(contours):
    epsilon = 0.05*cv2.arcLength(cnt,True)
    approxCnt.append(cv2.approxPolyDP(cnt,epsilon,True))

In [30]:
squares = [cnt for cnt in approxCnt if len(cnt) = 4]

In [19]:
contImg = np.zeros((480,640,3), dtype="uint8")
colors = np.random.randint(0,255,(len(contours),3))
color = [tuple([int(c) for c in color]) for color in colors]
for i, cnt in enumerate(squares):
     cv2.drawContours(contImg, squares, i, color[i])
cv2.imshow('contImg', contImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [52]:
color = np.random.randint(0,255,(len(contours),3))
tuple(color[1])

(249, 155, 113)

In [108]:
len(contours)
[len(cnt) for cnt in contours]

[2,
 20,
 36,
 17,
 31,
 25,
 9,
 155,
 479,
 132,
 196,
 208,
 193,
 190,
 51,
 56,
 35,
 34,
 25,
 25,
 50,
 46,
 36,
 36,
 25,
 22,
 51,
 52,
 35,
 34,
 19,
 23,
 423]

In [110]:
[len(cnt) for cnt in approxCnt if len(cnt) <= 4]

[2, 4, 2, 2]

In [81]:
(contours[top][25])[0]

array([195, 104], dtype=int32)

In [ ]:
point_x

In [63]:
tempL = get_vertices(contours, top, slope, tempL)

TypeError: only size-1 arrays can be converted to Python scalars

In [78]:
contours[top][0][0]

array([193, 102], dtype=int32)